# Exercise 2

First importing all the libraries required for Spark session in python. A SparkContext represents the connection to a Spark cluster, and is used to create RDDs, accumulators and broadcast variables on that cluster. The sql function on a SQLContext enables applications to run SQL queries programmatically and returns the result as a DataFrame .

In [1]:
import pyspark
from pyspark import SparkContext
sc =SparkContext()
from pyspark.sql import Row
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)
from pyspark.sql import SparkSession

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/03 08:32:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/07/03 08:32:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/07/03 08:32:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/07/03 08:32:13 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/07/03 08:32:13 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/07/03 08:32:13 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/07/03 08:32:13 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/07/03 08:32:13 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.


/opt/homebrew/lib/python3.9/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


A SparkSession can be used create DataFrame, register DataFrame as tables, execute SQL over tables, cache tables, and read data files. Class builder is Builder for SparkSession. appName() sets a name for the application, which will be shown in the Spark web UI. If no application name is set, a randomly generated name will be used. getOrCreate() gets an existing SparkSession or, if there is no existing one, creates a new one based on the options set in this builder.

Using spark.read.csv("path") we can read a CSV file with fields delimited by pipe, comma, tab (and many more) into a Spark DataFrame, These methods take a file path to read from as an argument.

In [2]:
spark = SparkSession.builder.appName(
    'Read CSV File into DataFrame').getOrCreate()
 
data = spark.read.csv('ml-10M100K-2/tags.dat', sep='||',inferSchema=True, header=True)

# Show the first 5 values of the dataframe
data.show(5)

22/07/03 08:32:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+------+-------+---------------+----------+
|UserID|MovieID|            Tag| Timestamp|
+------+-------+---------------+----------+
|    15|   4973|     excellent!|1215184630|
|    20|   1747|       politics|1188263867|
|    20|   1747|         satire|1188263867|
|    20|   2424|chick flick 212|1188263835|
|    20|   2424|          hanks|1188263835|
+------+-------+---------------+----------+
only showing top 5 rows



In [3]:
from pyspark import SparkFiles
from pyspark.sql.functions import col

In [4]:
# Ordering or Sorting the dataframe base din Timestamp column 
data.orderBy(col("Timestamp").asc()).show(10)

+------+-------+--------------------+----------+
|UserID|MovieID|                 Tag| Timestamp|
+------+-------+--------------------+----------+
| 19106|   1247|       mrs. robinson|1135313387|
| 19106|    145|           car chase|1135313403|
| 19106|   3481|          jack black|1135313411|
| 19106|   3481|               funny|1135313411|
| 19106|  34405|the man they call...|1135313453|
| 19106|   1396|    too many secrets|1135313492|
| 71331|   1396|     setec astronomy|1135361580|
| 22198|   2788|        monty python|1135429210|
| 22198|   1732|       coen brothers|1135429236|
| 22198|   1206|     stanley kubrick|1135429248|
+------+-------+--------------------+----------+
only showing top 10 rows



In [8]:
# In order to convert timestamp to readable date we import this in built library 
from datetime import datetime


In this function I take my timestamp value from the dataframe which is in string and convert it into integer. After that I use datetime.utcfromtimestamp().strftime() function to convert the time stamp into a format as day-month-Year Hours:Minutes:Seconds. Then I return this readable date.

In [9]:
def date_time(timestamp):
    timesatmp = int (timestamp)
    date = datetime.utcfromtimestamp(timesatmp).strftime('%d-%m-%Y %H:%M:%S')
    return date                                                 

In [10]:
date_time("1135429236")

'24-12-2005 13:00:36'

In [11]:
from pyspark.sql import functions as F
from pyspark.sql.functions import to_date
from pyspark.sql.types import StringType

In the code below I have taken the Timestamp column and performed the timestamp to date conversion by calling my user defined program date_time and saved the returned value from the function in a separate column named as Date_Time. We can see that we have obtained a new coolumn which contains the new readable date and time.

In [12]:
data1 = data.withColumn("Date_Time",  F.udf(date_time, StringType())(F.col("Timestamp")))
data1.show()

+------+-------+--------------------+----------+-------------------+
|UserID|MovieID|                 Tag| Timestamp|          Date_Time|
+------+-------+--------------------+----------+-------------------+
|    15|   4973|          excellent!|1215184630|04-07-2008 15:17:10|
|    20|   1747|            politics|1188263867|28-08-2007 01:17:47|
|    20|   1747|              satire|1188263867|28-08-2007 01:17:47|
|    20|   2424|     chick flick 212|1188263835|28-08-2007 01:17:15|
|    20|   2424|               hanks|1188263835|28-08-2007 01:17:15|
|    20|   2424|                ryan|1188263835|28-08-2007 01:17:15|
|    20|   2947|              action|1188263755|28-08-2007 01:15:55|
|    20|   2947|                bond|1188263756|28-08-2007 01:15:56|
|    20|   3033|               spoof|1188263880|28-08-2007 01:18:00|
|    20|   3033|           star wars|1188263880|28-08-2007 01:18:00|
|    20|   7438|              bloody|1188263801|28-08-2007 01:16:41|
|    20|   7438|             kung 

1. A tagging session for a user can be defined as the duration in which he/she generated tagging activities. Typically, an inactive duration of 30 mins is considered as a termination of the tagging session. Your task is to separate out tagging sessions for each user.

In [14]:
from pyspark.sql import Window

PYSPARK LAG is a function in PySpark that works as the offset row returning the value of the before row of a column with respect to the current row in PySpark. This is an operation in PySpark that returns the row just before the current row. Here I have used lag in the timestamp column and partitioned by the userIds. Here I get the previous timestamp for each userid.

In [28]:
data1 = data1.withColumn(
    "session_id",
    F.lag("Timestamp",1).over(Window.partitionBy("UserID").orderBy("Timestamp")),
)

In [29]:
data1.show(10)

+------+-------+---------------+----------+-------------------+----------+
|UserID|MovieID|            Tag| Timestamp|          Date_Time|session_id|
+------+-------+---------------+----------+-------------------+----------+
|    20|   2947|         action|1188263755|28-08-2007 01:15:55|      null|
|    20|   2947|           bond|1188263756|28-08-2007 01:15:56|1188263755|
|    20|   7438|         bloody|1188263801|28-08-2007 01:16:41|1188263756|
|    20|   7438|        kung fu|1188263801|28-08-2007 01:16:41|1188263801|
|    20|   7438|      Tarantino|1188263801|28-08-2007 01:16:41|1188263801|
|    20|   2424|chick flick 212|1188263835|28-08-2007 01:17:15|1188263801|
|    20|   2424|          hanks|1188263835|28-08-2007 01:17:15|1188263835|
|    20|   2424|           ryan|1188263835|28-08-2007 01:17:15|1188263835|
|    20|   1747|       politics|1188263867|28-08-2007 01:17:47|1188263835|
|    20|   1747|         satire|1188263867|28-08-2007 01:17:47|1188263867|
+------+-------+---------

Define if the session is the 1st one (more than 1800s after the previous one). Here I subtract the timestamps from the timestamp column and session id column created and check it it is less than 1800 seconds or 30 minutes. If it is less then I assign it as 0 or else I assign it as 1.


In [30]:
df = data1.withColumn("session_id",
    F.when(F.col("Timestamp") - F.col("session_id") <= 1800, 0).otherwise(1),
)

In [31]:
df.show(10)

+------+-------+---------------+----------+-------------------+----------+
|UserID|MovieID|            Tag| Timestamp|          Date_Time|session_id|
+------+-------+---------------+----------+-------------------+----------+
|    20|   2947|         action|1188263755|28-08-2007 01:15:55|         1|
|    20|   2947|           bond|1188263756|28-08-2007 01:15:56|         0|
|    20|   7438|         bloody|1188263801|28-08-2007 01:16:41|         0|
|    20|   7438|        kung fu|1188263801|28-08-2007 01:16:41|         0|
|    20|   7438|      Tarantino|1188263801|28-08-2007 01:16:41|         0|
|    20|   2424|chick flick 212|1188263835|28-08-2007 01:17:15|         0|
|    20|   2424|          hanks|1188263835|28-08-2007 01:17:15|         0|
|    20|   2424|           ryan|1188263835|28-08-2007 01:17:15|         0|
|    20|   1747|       politics|1188263867|28-08-2007 01:17:47|         0|
|    20|   1747|         satire|1188263867|28-08-2007 01:17:47|         0|
+------+-------+---------

Here I find out the unique session id based on the above result. So basically I sum over the session Id column and partiotion by User Id and again order by the timestamp column. Note that we can get same ids for different users.

In [32]:
# create a unique id per session
df = df.withColumn(
    "session_id",
    F.sum("session_id").over(Window.partitionBy("userid").orderBy("timestamp")),
)

In [33]:
df.show(10)

+------+-------+---------------+----------+-------------------+----------+
|UserID|MovieID|            Tag| Timestamp|          Date_Time|session_id|
+------+-------+---------------+----------+-------------------+----------+
|    20|   2947|         action|1188263755|28-08-2007 01:15:55|         1|
|    20|   2947|           bond|1188263756|28-08-2007 01:15:56|         1|
|    20|   7438|         bloody|1188263801|28-08-2007 01:16:41|         1|
|    20|   7438|        kung fu|1188263801|28-08-2007 01:16:41|         1|
|    20|   7438|      Tarantino|1188263801|28-08-2007 01:16:41|         1|
|    20|   2424|chick flick 212|1188263835|28-08-2007 01:17:15|         1|
|    20|   2424|          hanks|1188263835|28-08-2007 01:17:15|         1|
|    20|   2424|           ryan|1188263835|28-08-2007 01:17:15|         1|
|    20|   1747|       politics|1188263867|28-08-2007 01:17:47|         1|
|    20|   1747|         satire|1188263867|28-08-2007 01:17:47|         1|
+------+-------+---------

In the code below I finally create a unique id per session per user where F.dense_rank() returns the rank of rows within a window partition without any gaps.

In [34]:
df = df.withColumn(
    "session_id", F.dense_rank().over(Window.orderBy("userid", "session_id"))
)

In [36]:
df.show(10)

22/07/03 10:19:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:19:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:19:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/07/03 10:19:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:19:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:19:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/07/03 10:19:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:19:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:19:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+------+-------+---------------+----------+-------------------+----------+
|UserID|MovieID|            Tag| Timestamp|          Date_Time|session_id|
+------+-------+---------------+----------+-------------------+----------+
|    15|   4973|     excellent!|1215184630|04-07-2008 15:17:10|         1|
|    20|   2947|         action|1188263755|28-08-2007 01:15:55|         2|
|    20|   2947|           bond|1188263756|28-08-2007 01:15:56|         2|
|    20|   7438|         bloody|1188263801|28-08-2007 0

2. Once you have all the tagging sessions for each user, calculate the frequency of tagging for each user session.

In [37]:
# Here I have obtained the frequency of tagging for each user session
df1 = df.groupBy(['UserID','session_id']).count()
df1.show()

22/07/03 10:19:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:19:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:19:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:19:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:19:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:19:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+------+--

3. Find a mean and standard deviation of the tagging frequency of each user.

In [40]:
df2 = df1.groupBy('UserID','session_id').agg({'count': 'mean'})
df2.show()

22/07/03 10:20:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:20:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:20:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:20:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:20:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:20:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+------+--

In [41]:
 df3 = df1.groupBy('UserID').agg({'session_id': 'stddev'}).show()

22/07/03 10:21:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:21:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:21:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:21:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:21:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:21:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+------+--

4. Find a mean and standard deviation of the tagging frequency for across users.

In [42]:
mean = df1.agg({'count': 'mean'}).collect()
print("The mean across all users = ",mean[0][0])

22/07/03 10:21:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:21:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:21:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:21:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:21:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:21:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
The mean a

In [43]:
stddev = df1.agg({'count': 'stddev'}).collect()
print("The standard deviation across all users = ",stddev[0][0])

22/07/03 10:21:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:21:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:21:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:21:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:21:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:21:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
The standa

5. Provide the list of users with a mean tagging frequency within the two standard deviation from the mean frequency of all users.

In [45]:
# Range of two standard deviation from mean frequency of all users
std_1 = mean[0][0]+2*stddev[0][0]
std_2 = mean[0][0]-2*stddev[0][0]

Here I check if the tagging frequency lies between the above range of values. If it does then I assign it as 1 or else I assign it as 0

In [46]:
df1 = df1.withColumnRenamed("count","Frequency")
from pyspark.sql.functions import when
df4 = df1.withColumn("count", when(df1.Frequency<std_1, when(df1.Frequency>std_2,1).otherwise(0)).otherwise(0))
df4.show(100)

22/07/03 10:26:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:26:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:26:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:26:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:26:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:26:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+------+--

In [47]:
df4 = df4.withColumnRenamed("count","Users_in_the_range")
df5 = df4.filter(df4.Users_in_the_range == 1)

#### this dataframe df5 contains all the users whose mean frequency lies within 2 standard devaitions away from mean frequency of all users.

In [48]:
df5.show(10) 

22/07/03 10:27:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:27:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:27:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:27:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:27:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/03 10:27:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+------+--